# How manage volume of data 

In [68]:
import pandas as pd
import gc
s_pathFile = "../datos/sap/SAPx58_SYS_TABLE_COLUMNS.csv"
s_PickleFile = "../datos/sap/SAPx58_SYS_TABLE_COLUMNS.pickle"
s_ParqueteFile = "../datos/sap/SAPx58_SYS_TABLE_COLUMNS.parquete"
## 196M of size
#  -rw-r--r-- 1 krump krump 196M May 16 21:02 SAPD58_SYS_TABLE_COLUMNS.csv

# Part 1 - Basic load , Clasic CSV Load & Other formats as reference

## Using Low Memory = False for low latency

In [60]:
%%time
df_col_tables = pd.read_csv( s_pathFile , index_col=0, low_memory = False)

CPU times: user 1.38 s, sys: 190 ms, total: 1.57 s
Wall time: 1.57 s


#### Sample CSV vs Parquete

In [65]:
%time df_col_tables.to_pickle(s_ParqueteFile)
%time df_col_tables = pd.read_pickle(s_ParqueteFile)

CPU times: user 157 ms, sys: 149 ms, total: 306 ms
Wall time: 306 ms
CPU times: user 196 ms, sys: 61.8 ms, total: 257 ms
Wall time: 257 ms


#### Sample CSV vs Pickle

In [66]:
%time df_col_tables.to_pickle(s_PickleFile)
%time df_col_tables = pd.read_pickle(s_PickleFile)

CPU times: user 204 ms, sys: 94.8 ms, total: 299 ms
Wall time: 298 ms
CPU times: user 180 ms, sys: 69.9 ms, total: 250 ms
Wall time: 250 ms


## Using Python Engine option, It Takes more time

In [8]:
%%time
df_col_tables = pd.read_csv( s_pathFile , index_col=0, engine='python')


CPU times: user 10.2 s, sys: 380 ms, total: 10.6 s
Wall time: 10.6 s


## **Conclusion**

Using the next settings with **low_memory = False** can result in a better time execution

    df_col_tables = pd.read_csv( s_pathFile , index_col=0, low_memory = False)\

Using the next settings with **lengine='python'** takes more time, also is not possible to combine options

    df_col_tables = pd.read_csv( s_pathFile , index_col=0, engine='python')

**notes**

But in comparison pickle and parquete format are more efficient in terms of the read processes  but not necessary in storage process

* Parquete normally is used for long term storage
* Pickle normally I used for quick and short storage

rw-r--r-- 1 krump krump 196M May 16 21:02 SAPX58_SYS_TABLE_COLUMNS.csv
rw-r--r-- 1 krump users 278M Jun 25 13:32 SAPX58_SYS_TABLE_COLUMNS.parquete-rw-r--r-- 1 krump users 278M Jun 25 13:32 SAXD58_SYS_TABLE_COLUMNS.pickle


In [67]:
%%time df_col_tables

UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


### Drop all rows

In [32]:
df_col_tables.drop(df_col_tables.index, inplace=True)
df_col_tables
gc.collect()

29

# Part 2 load in parts

This could take more time, but consider the file split and possible process in parts instead of big block of data

In [42]:
%%time
part = 0 
for chunk in pd.read_csv( s_pathFile , index_col=0, engine='python',  chunksize = 10000 , iterator=True):
    #print('part ', part , len(chunk))
    df_col_tables = pd.concat([df_col_tables, chunk] )
    #part = part + 1

CPU times: user 1min 2s, sys: 36.5 s, total: 1min 39s
Wall time: 1min 39s


In [34]:
%%time
df_col_tables

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


,SCHEMA_NAME,TABLE_NAME,TABLE_OID,COLUMN_NAME,POSITION,DATA_TYPE_ID,DATA_TYPE_NAME,OFFSET,LENGTH,SCALE,...,IS_CACHE_KEY,ROW_ORDER_POSITION,IS_HIDDEN,IS_MASKED,MASK_EXPRESSION,CLIENTSIDE_ENCRYPTION_STATUS,CLIENTSIDE_ENCRYPTION_COLUMN_KEY_ID,CLIENTSIDE_ENCRYPTION_MODE,PERSISTENT_MEMORY,NUMA_NODE_INDEXES
0,SYS,AFL_PACKAGES_,131325,FILE,8,-10,NCLOB,120,2147483647,NaN,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,SYS,P_USER_PASSWORD_,131422,ADMIN_GIVEN_PASSWORD,4,-6,TINYINT,97,3,0.0,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,SYS,ABSTRACT_SQL_PLAN_DATA_,26563578,MIN_EXECUTION_TIME_NOT_USING_ABSTRACT_SQL_PLAN,15,-5,BIGINT,133,19,0.0,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,SAPD58,CATK,196799,R2MAND,25,-9,NVARCHAR,8,3,NaN,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,_SYS_STATISTICS,STATISTICS_ALERT_THRESHOLDS_PROFILES,18972719,SEVERITY,3,-6,TINYINT,8,3,0.0,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119623,SAPD58,/BIC/B0016469000,16223457,RECORD,4,4,INTEGER,8,10,0.0,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1119624,SAPD58,/BIC/B0016469000,16223457,REQUEST,1,-9,NVARCHAR,16,30,NaN,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1119625,SAPD58,/BIC/B0016469000,16223457,TXTLG,9,-9,NVARCHAR,16,60,NaN,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1119626,SAPD58,/BIC/B0016469000,16223457,TXTMD,8,-9,NVARCHAR,16,40,NaN,...,False,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN


# load using multi Threath

In [52]:
s_pathFile = "../datos/sap/SAPX58_TPFET.csv"

In [ ]:
from multiprocessing.pool import ThreadPool as Pool
from random import randint
from time import sleep


def process_line(l):
    print (l, "started")
    sleep(randint(0, 3))
    print (l, "done")


def get_next_line():
    with open(s_pathFile, 'r') as f:
        for line in f:
            yield line

f = get_next_line()
t = Pool(processes=8)

for i in f:
    t.map(process_line, (i,))
t.close()
t.join()